open-meteo.com provides hourly weather data, with high resolution. Hourly updates. predictions 16 days into to the future, as well as historical data.. Here the features are explored...

You can access past weather data dating back to 1940 with the historical weather API 
offered. *However, there is a 5-day delay in the data*. If you want information for the most recent days, you can use the forecast API and adjust the Past Days setting.

https://open-meteo.com/en/docs/historical-weather-api

The strongest correlation to PV (photovoltaic) production is generally found with the following historical weather features:
1.	Diffuse Horizontal Irradiance (DHI)
2.	Direct Normal Irradiance (DNI) – This measures the amount of solar radiation received per unit area directly from the sun at a normal (perpendicular) angle. It is crucial for systems with solar tracking but less relevant for fixed panels.
3.	Shortwave Solar Radiation (GHI) – Global Horizontal Irradiance (GHI) is the total solar radiation received on a horizontal surface and is commonly used in PV modeling.

Additional weaterh features selected for download:

4.Cloud cover Total

5.Windspeed (10m) - due to cooling effect on PVs & wind energy production

6.Temperature (2 m) - due to effect on PV efficiency and consumption level

7.wind speed (100 m) for wind energy production

     Data collected for De Bilt, Eelde (Groningen), Vlissingen, Sittard.
     for period 2022-2025March.

In [1]:
%pip install openmeteo-requests
%pip install requests-cache retry-requests numpy pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
from datetime import datetime, timezone
import os

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define locations
locations = [
    ("DeBilt", 52.12949, 5.20514),
    ("Eelde", 53.214469, 6.566481),
    ("Vlissingen", 51.455677, 3.576488),
    ("Sittard", 50.998299, 5.86291)
]

# Open-Meteo API parameters
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": [loc[1] for loc in locations],
    "longitude": [loc[2] for loc in locations],
    "start_date": "2022-01-01",
    "end_date": "2025-03-31",
    "hourly": ["temperature_2m", "apparent_temperature", "cloud_cover", "wind_speed_10m", "diffuse_radiation", "direct_normal_irradiance", "shortwave_radiation", "wind_speed_100m"]
}

# Fetch data from Open-Meteo
responses = openmeteo.weather_api(url, params=params)

# Store results in a list
dataframes = []
for i, response in enumerate(responses):
    location_name = locations[i][0]
    
    # Process hourly data
    hourly = response.Hourly()
    hourly_data = {
        "date": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
        "apparent_temperature": hourly.Variables(1).ValuesAsNumpy(),
        "cloud_cover": hourly.Variables(2).ValuesAsNumpy(),
        "wind_speed_10m": hourly.Variables(3).ValuesAsNumpy(),
        "diffuse_radiation": hourly.Variables(4).ValuesAsNumpy(),
        "direct_normal_irradiance": hourly.Variables(5).ValuesAsNumpy(),
        "shortwave_radiation": hourly.Variables(6).ValuesAsNumpy(),
        "wind_speed_100m": hourly.Variables(7).ValuesAsNumpy(),
        "location": location_name  # Add location column
    }

    # Convert to DataFrame
    df = pd.DataFrame(hourly_data)
    dataframes.append(df)

# Combine all data into a single DataFrame
final_dataframe = pd.concat(dataframes, ignore_index=True)

# Generate timestamp for filename
fetch_timestamp = datetime.now(timezone.utc)
timestamp_str = fetch_timestamp.isoformat(timespec='seconds').replace(':', '-')



csv_filename = f"../Data/hist_weather_data_2022-2025Mar31_{timestamp_str}.csv"
final_dataframe.to_csv(csv_filename, index=False)

print(f"Data saved to {csv_filename}")

Data saved to ../Data/hist_weather_data_2022-2025Mar31_2025-04-14T07-16-22+00-00.csv


In [3]:
most_recent_date = final_dataframe['date'].max()
print(f"The most recent date in the dataframe is: {most_recent_date}")

The most recent date in the dataframe is: 2025-03-31 23:00:00+00:00


In [4]:
# Calculate the memory usage of the dataframe in bytes
memory_usage_bytes = final_dataframe.memory_usage(deep=True).sum()

# Convert the memory usage to kilobytes
memory_usage_kb = memory_usage_bytes / 1024

print(f"The size of the dataframe in kilobytes is: {memory_usage_kb:.2f} KB")

The size of the dataframe in kilobytes is: 10674.13 KB


In [5]:
# Generate descriptive statistics for all columns in the dataframe
descriptive_stats = final_dataframe.describe()
print(descriptive_stats)

       temperature_2m  apparent_temperature    cloud_cover  wind_speed_10m  \
count   113856.000000         113856.000000  113856.000000   113856.000000   
mean        11.075655              8.549459      68.543205       14.466681   
std          6.642730              8.028013      39.177685        7.477279   
min         -9.667500            -14.067160       0.000000        0.000000   
25%          6.306500              2.504193      30.000000        8.707238   
50%         10.667500              7.637471      97.000000       13.237038   
75%         15.856501             14.482809     100.000000       18.875126   
max         38.667500             38.013386     100.000000       58.288883   

       diffuse_radiation  direct_normal_irradiance  shortwave_radiation  \
count      113856.000000             113856.000000        113856.000000   
mean           52.544224                150.969879           128.530136   
std            75.161232                246.501144           201.683990 